# Fetch data from CoinGecko

In [6]:
import numpy as np 
import matplotlib.pyplot as plt

In [7]:
#!/usr/bin/env python3
import requests
import datetime
from sqlalchemy import (
    create_engine, Column, Integer, Date, Float
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [8]:
# Configuration
API_URL = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"

In [9]:
### Sanity checks
full = requests.get(API_URL, params={"vs_currency":"usd","days":5})
print(full.url)       # shows the expanded URL
print(full.status_code)
print(full.json().keys())

https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=5
200
dict_keys(['prices', 'market_caps', 'total_volumes'])


In [24]:
### Set up and Then Create a DataBase
DB_PATH = "../data/btc.sqlite"
# Set up SQLAlchemy
Base = declarative_base()

/tmp/ipykernel_5604/2864823468.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [25]:
Base.metadata, Base.mro(), Base.registry

(MetaData(),
 [sqlalchemy.orm.decl_api.Base, object],
 <sqlalchemy.orm.decl_api.registry at 0x70f0cca775f0>)

In [26]:
## Define a class for the entries.
## Each row will be a PricePoint with certain attributes
class PricePoint(Base):
    __tablename__ = "price_points"
    id = Column(Integer, primary_key=True)
    date = Column(Date, unique=True, nullable=False)
    price = Column(Float, nullable=False)

In [36]:
_pp = PricePoint(date="2025-06-25", price=15)
_pp.id, _pp.__tablename__, _pp.date, _pp.price

(None, 'price_points', '2025-06-25', 15)

## Create the database and session. 

In [ ]:
engine = create_engine(f"sqlite:///{DB_PATH}")
Session = sessionmaker(bind=engine)

In [ ]:
def fetch_last_5_days():
    today = datetime.date.today()
    # CoinGecko takes days as number; get 5 days back
    params = {"vs_currency": "usd", "days": 5}
    resp = requests.get(API_URL, params=params)
    resp.raise_for_status()
    data = resp.json()
    # data["prices"] is a list of [timestamp_ms, price]
    points = []
    for ts_ms, price in data["prices"]:
        # Convert ms → date
        dt = datetime.date.fromtimestamp(ts_ms / 1000)
        points.append((dt, float(price)))
    # Keep unique dates (API returns multiple per day)
    unique = {}
    for dt, price in points:
        unique[dt] = price
    return unique.items()

In [11]:
# create table if needed
Base.metadata.create_all(engine)
session = Session()

In [12]:
for dt, price in fetch_last_5_days():
    # upsert: skip if already present
    exists = session.query(PricePoint).filter_by(date=dt).first()
    if exists:
        exists.price = price
        print(f"Updated {dt}: {price}")
    else:
        pp = PricePoint(date=dt, price=price)
        session.add(pp)
        print(f"Inserted {dt}: {price}")
# session.commit()
# session.close()
# print("Done.")

Updated 2025-06-21: 102857.80399902674
Updated 2025-06-22: 99514.84004623022
Updated 2025-06-23: 103770.17564882364
Updated 2025-06-24: 106091.85705647874
Updated 2025-06-25: 107822.58717700628
Updated 2025-06-26: 107104.91205849432


In [16]:
session = Session()                   # open a transactional context
# 

In [17]:
pp = PricePoint(date=dt, price=price)
# session.add(pp)                       # stage a new row
# session.commit()                      # flushes SQL to the DB
# session.close()


https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=5
200
dict_keys(['prices', 'market_caps', 'total_volumes'])


sessions are just units of work, and you can certainly keep one open for a series of related operations. That said, there are good reasons to open and close sessions more granularly:

Why close (and reopen) sessions
Transaction boundaries
Each session manages its own transaction. By committing and closing, you demarcate “this batch of work is done.” If you kept the session open forever, you’d be in one long-running transaction—risking stale data or locks.

Resource management
A session holds onto identity maps, caches, and database connections. Closing frees those resources so they can be reused elsewhere.

Error isolation
If something goes wrong (an exception), you can roll back that one session without impacting other work.